## 7日目
* 昨日は for 構文、 Writer モナド 、関数をモナドとして扱う Readerモナドを見た
* 正直まだモナドが何かわかってはいない

In [1]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

### Applicative Builder

In [2]:
(3.some |@| 5.some) {_ + _}

res1: Option[Int] = Some(8)

In [5]:
(3.some |@| 5.some) {_ * _}

res4: Option[Int] = Some(15)

In [6]:
val f = ({(_: Int) * 2} |@| {(_:Int) + 10}) {_ + _}

f: Int => Int = <function1>

In [8]:
f(3)

res7: Int = 19

* `|@|` この演算子、別名マコーレーカルキン演算子というらしい。 |@|
    - http://akiradeveloper.hatenadiary.com/entry/2015/06/24/093931

In [9]:
type Stack = List[Int]

defined type Stack

In [10]:
def pop(stack: Stack): (Int, Stack) = stack match {
    case x :: xs => (x, xs)
}

defined function pop

In [12]:
pop(List(1, 2, 3))

res11: (Int, Stack) = (1, List(2, 3))

In [13]:
def push(a: Int, stack: Stack): (Unit, Stack) = ((), a :: stack)

defined function push

In [14]:
push(1, List(3,4))

res13: (Unit, Stack) = ((), List(1, 3, 4))

In [15]:
def stackManip(stack: Stack): (Int, Stack) = {
    val (_, newStack1) = push(3, stack)
    val (a, newStack2) = pop(newStack1)
    pop(newStack2)
}

defined function stackManip

In [16]:
stackManip(List(5, 8, 2, 1))

res15: (Int, Stack) = (5, List(8, 2, 1))

* これは State モナドの例なの？

### State and StateT
* 状態付きの計算とは、ある状態をとって、更新された状態と一緒に計算結果を返す関数として表現できる

```scala
s -> (a, s)
```

* なるほど、さっきのStackは確かに値とStack(状態)とを返していた

```scala
type State[S, +A] = StateT[Id, S, A]

object State extends StateFuncitons {
    def apply[S, A](f: S => (S, A)): State[S, A] = new StateT[Id, S, A] {
        def apply(s: S) = f(s)
    }
}
```

```scala
trait StateT[F[+_], s, +A] { self =>
    def apply(initial: S): F[(S, A)]
    def run(initial: S): F[(S, A)] = apply(initial)
    def runZero(implicit S Monoid[S]): F[(S, A)] = 
    run(S.zero)
}
```

In [17]:
State[List[Int], Int] { case x :: xs => (xs, x) }

res16: State[List[Int], Int] = scalaz.IndexedStateT$$anon$10@403d3c2b

In [20]:
type Stack = List[Int]
val pop = State[Stack, Int] {
    case x :: xs => (xs, x)
}
def push(a: Int) = State[Stack, Unit] {
    case xs => (a :: xs, ())
}
def stackManip: State[Stack, Int] = for {
    _ <- push(3)
    a <- pop
    b <- pop
} yield(b)

defined type Stack
pop: State[Stack, Int] = scalaz.IndexedStateT$$anon$10@6682abb8
defined function push
defined function stackManip

In [21]:
stackManip(List(5, 8, 2, 1))

res20: (Stack, Int) = (List(8, 2, 1), 5)

* これ見ると for 構文の有用さがわかるかも

In [21]:
new State[List(5, 8, 2, 1), Unit].push(3).pop().pop()

: 

### 状態の取得と設定
* Control.Monad.State モジュールは、２つの便利な関数 get と push を備えた、MonadState という型クラスを提供している

* State object は `StateFunctions trait` を継承して、いくつかのヘルパー関数を定義する

```scala
trait StateFunction {
    def constantState[S, A](a: A, s: => S): State[S, A] =
        State((_: S) => (s, a))
    def State[S, A](a: A): State[S, A] =
        State((_: S, a))
    def init[S]: State[S, S] = State(s => (s, s))
    def get[S]: State[S, S] = init
    def gets[S, T](f: S => T): State[S, T] = State(s => (s, f(s)))
    def put[S](s: S): State[S, Unit] = State(_ => (s, ()))
    def modify[S](f: S => S): State[S, Unit] = State(s => {
        val r = f(s);
        (r, ())
    })
    
    def delta[A](a: A)(implicit A: Group[A]): State[A, A] = State {
        (prevA) =>
            val diff = A.minus(a, prevA)
            (diff, a)
    }
}
```

In [22]:
def init[S]: State[S, S] = State(s => (s, s))
def get[S]: State[S, S] = init

defined function init
defined function get

In [23]:
def put[S](s: S): State[S, Unit] = State(_ => (s, ()))

defined function put

In [24]:
def stackyStack: State[Stack, Unit] = for {
    stackNow <- get
    r <- if (stackNow === List(1, 2, 3)) put(List(8, 3, 1))
        else put(List(9, 2, 1))
} yield r

defined function stackyStack

In [25]:
stackyStack(List(1, 2, 3))

res24: (Stack, Unit) = (List(8, 3, 1), ())

In [26]:
val pop: State[Stack, Int] = for {
    s <- get[Stack]
    val (x :: xs) = s
    _ <- put(xs)
} yield x

pop: State[Stack, Int] = scalaz.IndexedStateT$$anon$11@39f043e7

In [27]:
def push(x: Int): State[Stack, Unit] = for {
    xs <- get[Stack]
    r <- put(x :: xs)
} yield r

defined function push

### \/
* Eigher e a 型も失敗の文脈を与えるモナド
* しかも、失敗に値を付加できるので、何が失敗したかを説明したり、そのほか失敗にまつわる有用な情報を提供できる

```scala
sealed trait \/[+A, +B] {
    ...
    def isLeft: Boolean =
      this match {
          case -\/(_) => true
          case \/-(_) => false
      }
    
    def isRight: Boolean =
      this match {
          case -\/(_) => false
          case \/-(_) => true
      }
    ...
    def swap: (B \/ A) =
      this match {
          case -\/(a) => \/-(a)
          case \/-(b) => -\/(b)
      }
    
    def unary_~ : (B \/ A) = swap
    ...
    def getOrElse[BB >: B](x: => BB): BB =
      toOption getOrElse x
    
    def |[BB >: B](x: => BB): BB = getOrElse(x)
    
    def orElse[AA >: A, BB >: B](x: => AA \/ BB): AA \/ BB =
      this match {
          case -\/(_) => x
          case \/-(_) => this
      }
    
    def |||[AA >: A, BB >: B](x: => AA \/ BB): AA \/ BB = orElse(x)
    ...
}

private case class -\/[+A](a: A) extends (A \/ Nothing)
private case class \/-[+B](b: B) extends (Nothing \/ B)
```

In [28]:
1.right[String]

res27: String \/ Int = \/-(1)

In [29]:
"error".left[Int]

res28: String \/ Int = -\/("error")

In [29]:
Left[String, Int]("boom") flatMap { x => Right[String, Int](x + 1) }

cmd29.sc:1: value flatMap is not a member of scala.util.Left[String,Int]
val res29 = Left[String, Int]("boom") flatMap { x => Right[String, Int](x + 1) }
                                      ^

: 

In [30]:
Left[String, Int]("boom").right flatMap { x => Right[String, Int](x + 1) }

res29: Either[String, Int] = Left("boom")

In [31]:
"boom".left[Int] >>= { x => (x + 1).right }

res30: String \/ Int = -\/("boom")

In [32]:
for {
    e1 <- "event 1 ok".right
    e2 <- "event 2 failed!".left[String]
    e3 <- "event 3 failed!".left[String]
} yield (e1 |+| e2 |+| e3)

res31: String \/ String = -\/("event 2 failed!")

In [33]:
"event 1 ok".right.isRight

res32: Boolean = true

In [34]:
"event 1 ok".right.isLeft

res33: Boolean = false

In [35]:
"event 1 ok".right | "something bad"

res34: String = "event 1 ok"

In [36]:
~"event 2 failed!".left[String] | "something good"

res35: String = "event 2 failed!"

In [37]:
"event 1 ok".right map {_ + "!"}

res36: Nothing \/ String = \/-("event 1 ok!")

* 左側で連鎖させるには、 => AA \/ BB (ただし [AA >: A, BB >: B]) を受け取る orElse がある
* orElse のシンボルを使ったエイリアスは |||

In [38]:
"event 1 failed!".left ||| "retry event 1 ok".right

res37: Nothing \/ String = \/-("retry event 1 ok")

### Validation

```scala
sealed trait Validation[+E, +A] {
    def isSuccess: Boolean = this match {
        case Success(_) => true
        case Failure(_) => false
    }
    
    def isFailure: Boolean = !isSuccess
    
    ...
}

final case class Success[E, A](a: A) extends Valication[E, A]
final case class Failure[E, A](e: E) extends Validation[E, A]
```

In [39]:
"event 1 ok".success[String]

res38: Validation[String, String] = Success("event 1 ok")

In [40]:
"event 1 failed!".failure[String]

res39: Validation[String, String] = Failure("event 1 failed!")

* Validation はモナドではなく Applicative functor
* 最初のイベントの結果を次へ連鎖するのではなく、全イベントを検証する

In [41]:
("event 1 ok".success[String] |@| "event 2 failed!".failure[String] |@| "event 3 failed!".failure[String]) {_ + _ + _}

res40: Validation[String, String] = Failure("event 2 failed!event 3 failed!")

* 計算途中でショートさせた \/ と違って、 Validation は計算を続行して全ての失敗を報告する

### NonEmptyList

```scala
sealed trait NonEmptyList[+A] {
  val head: A
  val tail: List[A]
  def <::[AA >: A](b: AA): NonEmptyList[AA] = nel(b, head :: tail)]
  ...
}
```

In [42]:
1.wrapNel

res41: NonEmptyList[Int] = NonEmpty[1]

In [43]:
"event 1 ok".successNel[String]

res42: ValidationNel[String, String] = Success(event 1 ok)

In [44]:
"event 1 failed!".failureNel[String]

res43: ValidationNel[String, String] = Failure(NonEmpty[event 1 failed!])

In [45]:
("event 1 ok".successNel[String] |@| "event 2 failed!".failureNel[String] |@| "event 3 failed!".failureNel[String]) {_ + _ + _}

res44: Validation[NonEmptyList[String], String] = Failure(NonEmpty[event 2 failed!,event 3 failed!])

* Failure の中に全ての失敗メッセージを集約することができた